In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
%config Completer.use_jedi = False
import tqdm
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
#mpl.rcParams['figure.figsize'] = (6, 4)
import numpy as np
import h5py

%load_ext autoreload
%autoreload 2

In [27]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column, row

data = {'x':np.linspace(1e-3,1-1e-3,1000),'y':0.5*np.ones(1000)}
source = ColumnDataSource(data=data)

output_file('coinflipping.html')
p = figure(title='Beta distribution',plot_width=600, plot_height=600,
           background_fill_color="#fafafa")
p.y_range.start = 0.0
p.x_range.start = 0
p.x_range.end = 1
p.xaxis.axis_label = 'Bias p'
p.yaxis.axis_label = 'Beta(p | a, b)'

p.line('x', 'y', line_width=2,source=source)

# Add a slider to control which year the patches are coloured by
alpha_slider = Slider(start=-2, end=2, value=0.0, step=0.001, title="log10(a)")
beta_slider = Slider(start=-2, end=2, value=0.0, step=0.001, title="log10(b)")

# Add the custom javascript to interface the slider with the patches
callback = CustomJS(args=dict(source=source, alpha=alpha_slider, beta=beta_slider), code="""

        var g = 7;
        var C = [0.99999999999980993, 676.5203681218851, -1259.1392167224028,771.32342877765313, -176.61502916214059, 12.507343278686905, -0.13857109526572012, 9.9843695780195716e-6, 1.5056327351493116e-7];

        function gamma(z) {

            if (z < 0.5) return Math.PI / (Math.sin(Math.PI * z) * gamma(1 - z));
            else {
                z -= 1;

                var x = C[0];
                for (var i = 1; i < g + 2; i++)
                x += C[i] / (z + i);

                var t = z + g + 0.5;
                return Math.sqrt(2 * Math.PI) * Math.pow(t, (z + 0.5)) * Math.exp(-t) * x;
            }
        }
        
        var data = source.data;
        const new_alpha = Math.pow(10,alpha.value);
        const new_beta = Math.pow(10,beta.value);
        const x = data['x']
        const y = data['y']
        var B = gamma(new_alpha)*gamma(new_beta)/gamma(new_alpha+new_beta);
        for (var i = 0; i < x.length; i++) {
            y[i] = Math.pow(x[i],new_alpha-1.0)*Math.pow(1.0-x[i],new_beta-1.0)/B;
        }
    
        source.change.emit();
    """)
alpha_slider.js_on_change('value', callback)
beta_slider.js_on_change('value', callback)

# Combine plot and slider and output
layout = column(p,alpha_slider, beta_slider)
show(layout)
